In [ ]:
import numpy as np
import scipy
import time
import projections_POVM 
import timeit

## Data


In [ ]:
import pickle

# Load dictionary from file
with open('dataExactPOVM_p1e1.pkl', 'rb') as pickle_file:
    data_sdp = pickle.load(pickle_file)

In [ ]:
def generate_samples(data_sdp, qubits, iters, tol, reps, data):
    """
    Generate samples for quantum operations based on provided data and parameters.

    Args:
    - data_sdp (dict): Dictionary containing SDP data.
    - qubits (int): Number of qubits for the quantum operation.
    - iters (int): Number of iterations for iterative algorithms.
    - tol (float): Tolerance level for convergence criteria.
    - reps (int): Number of repetitions for sampling.
    - data (dict): Dictionary to store generated data.

    Returns:
    - data (dict): Dictionary containing generated data.

    This function iterates over the specified number of repetitions and performs quantum operations
    based on the provided data and parameters. It calculates distances and times for each operation
    compared to exact values and stores them in the data dictionary.
    """
    for k in range(0, reps):
        povmInput = np.array(data_sdp[f"data_{qubits}qubits"]['povmInput'][k])
        povmExact = np.array(data_sdp[f"data_{qubits}qubits"]['povmOutSDP'][k])
        machine = projections_POVM.closestPOVM(qubits, iters)
        timeTSE, povmTSE = machine.two_step_estimation(povmInput, Id = False)
        timeCBA, povmCBA = machine.cholesky_based_approximation(povmInput, pos = False)
        timeDykstraCBA, povmDykstraCBA, itersDykstraCBA = machine.dykstra_cholesky_based_approximation(povmInput, tol)
        timeDykstraTSE, povmDykstraTSE, itersDykstraTSE = machine.dykstra_two_step_approximation(povmInput, tol)
        data['povmCBA'][k] = np.linalg.norm(povmCBA-povmExact)
        data['timeCBA'][k] = timeCBA
        data['povmDykstraCBA'][k] = np.linalg.norm(povmDykstraCBA-povmExact)
        data['timeDykstraCBA'][k] = timeDykstraCBA
        data['itersDykstraCBA'][k] = itersDykstraCBA
        data['povmTSE'][k] = np.linalg.norm(povmTSE-povmExact)
        data['timeTSE'][k] = timeTSE
        data['povmDykstraTSE'][k] = np.linalg.norm(povmDykstraTSE-povmExact)
        data['timeDykstraTSE'][k] = timeDykstraTSE
        data['itersDykstraTSE'][k] = itersDykstraTSE

    return data 

In [ ]:
def generate_samples_qubits(data_sdp, qubits_list, iters, tol, reps):
    """
    Generate samples for quantum operations with varying qubit numbers.

    Args:
    - data_sdp (dict): Dictionary containing SDP data.
    - qubits_list (list): List of qubit numbers for quantum operations.
    - iters (int): Number of iterations for iterative algorithms.
    - tol (float): Tolerance level for convergence criteria.
    - reps (int): Number of repetitions for sampling.

    Returns:
    - data_qubits (dict): Dictionary containing generated data for each qubit configuration.

    This function iterates over a list of qubit numbers, generates data for each qubit configuration,
    and stores the results in a dictionary. It prints the progress as it runs for each qubit configuration.
    """
    data_qubits = {}
    for qubits in qubits_list: 
        print(f"Running for {qubits} qubits... ")
        data = {'povmCBA': [0 for _ in range(reps)], 'timeCBA':np.zeros((reps)), 'povmDykstraCBA': [0 for _ in range(reps)],'timeDykstraCBA': np.zeros((reps)),
                    'itersDykstraCBA': np.zeros((reps)),  'povmTSE':[0 for _ in range(reps)],'timeTSE': np.zeros((reps)), 'povmDykstraTSE': [0 for _ in range(reps)], 
                    'timeDykstraTSE':np.zeros((reps)), 'itersDykstraTSE':np.zeros((reps))} 
        data = generate_samples(data_sdp, qubits, iters, tol, reps, data)
        data_qubits[f'data_{qubits}qubits'] = data 

    return data_qubits  

In [ ]:
reps = 100
iters = 100
tol = 1e-7
max_qubits = 7
qubits_list = [x for x in range(3, max_qubits)]

data_qubits = generate_samples_qubits(data_sdp, qubits_list, iters, tol, reps)

In [ ]:
import pickle

# Save dictionary to a file
with open('dataApproxPOVM_p1e1.pkl', 'wb') as pickle_file:
    pickle.dump(data_qubits, pickle_file)
